In [69]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

In [70]:
def removeIdentifierFile(path):
    for file in os.listdir(path):
        if file.find(".Identifier") != -1:
            os.remove(os.path.join(path, file))

In [71]:
import os, random

raw_imposters_folder = "imposters"
raw_valid_user_folder = "valid_user"

removeIdentifierFile(raw_imposters_folder)
removeIdentifierFile(raw_valid_user_folder)

img_count = 500 # img taken from each class

imposter_filenames = random.sample(os.listdir(raw_imposters_folder), img_count)
valid_user_filenames = random.sample(os.listdir(raw_valid_user_folder), img_count)

train_valid_user_folder = "train/valid_user"
train_imposters_folder = "train/imposters"

Path(train_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(train_imposters_folder).mkdir(parents=True, exist_ok=True)

test_valid_user_folder = "test/valid_user"
test_imposters_folder = "test/imposters"

Path(test_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(test_imposters_folder).mkdir(parents=True, exist_ok=True)


In [72]:
class_training_size = int(img_count * 0.9)

count = 0
for imposter_filename in imposter_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(train_imposters_folder, imposter_filename))
    count += 1

count = 0
for imposter_filename in imposter_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(test_imposters_folder, imposter_filename))


for valid_user_filename in valid_user_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(train_valid_user_folder, valid_user_filename))

for valid_user_filename in valid_user_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(test_valid_user_folder, valid_user_filename))

removeIdentifierFile(train_imposters_folder)
removeIdentifierFile(test_imposters_folder)
removeIdentifierFile(train_valid_user_folder)
removeIdentifierFile(test_valid_user_folder)


450
50


In [73]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("train",
    target_size=(150,150),
    batch_size = 32,
    class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("test",
    target_size=(150,150),
    batch_size =32,
    class_mode = 'binary')

Found 900 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [74]:
test_dataset.class_indices


{'imposters': 0, 'valid_user': 1}

In [75]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512,activation='relu'))

model.add(keras.layers.Dense(1,activation='sigmoid'))

2023-01-21 12:41:52.578144: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-21 12:41:52.578836: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/magician3124/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-01-21 12:41:52.579707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/magician3124/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-01-21 12:41:52.579811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:6

In [76]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [78]:
model.fit_generator(train_dataset,
    steps_per_epoch = 250,
    epochs = 10,
    validation_data = test_dataset
)

model.save('classifier')

/tmp/ipykernel_9941/1593974248.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_dataset,


Epoch 1/10
250/250 [==============================] - 16s 60ms/step - loss: 0.0229 - accuracy: 0.9967 - val_loss: 0.0035 - val_accuracy: 1.0000


INFO:tensorflow:Assets written to: classifier/assets


INFO:tensorflow:Assets written to: classifier/assets
